In [1]:
import pathlib
import pandas as pd
import numpy as np
import re
import time
import fasttext
fasttext.FastText.eprint = lambda x: None
from huggingface_hub import hf_hub_download
from tqdm import tqdm

In [2]:
def get_lang(
    df: pd.DataFrame,
    col_calculate_on: str
) -> str:
    """Detects the language of a text column in a DataFrame using langdetect.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the text column
    col_calculate_on : str
        Column name to calculate language on

    Returns
    -------
    df : pd.DataFrame
        DataFrame with the language column added to it as "lang"
    """

    def det(x: str) -> str:
        """
        Detects the language of a given text

        Parameters
        ----------
        x : str
            Text whose language is to be detected

        Returns
        -------
        lang : str
            Language of the text
        """

        try:
            lang = model.predict(x)[0][0].replace('__label__','')
        except Exception as e:
            print(e)
            lang = 'Other'
        return lang

    print(f"-- Detecting language...")
    start_time = time.time()

    df['lang'] = df[col_calculate_on].apply(det)

    print(f'-- -- Language detect finished in {(time.time() - start_time)}')

    return df

def normalize_string(s):
    # Convert to lower case
    #s = s.lower()
    # Remove punctuation and special characters
    #s = re.sub(r'[^\w\s]', '', s)
    # Remove extra spaces
    s = re.sub(r'\s+', ' ', s).strip()
    return s

Estos ficheros se generan con el notebook "Procesado_Textos_BSC".

No obstante, algo no cuadra: 
- Montse dijo que había **155.980** documentos en **catalán**
- De los documentos que nos pasan, sólo aparecen **3.942** (esto lo obtenemos si leemos ```/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated``` y hacemos ``combined_df_ca[(combined_df_ca['translated_content'] != "") & (pd.notna(combined_df_ca['translated_content']))]`` o ``combined_df_ca[combined_df_ca.alternative_lang == "ca"]``. 
- Si nos quedamos con los que son ténicos, sólo quedan **839** documentos traducidos del catalán. ¿No son pocos?


**Repuesta de BSC de cómo detectan el lenguaje:** Para la identificación de idiomas de nextProcurement usamos el modelo de Fasttext: https://fasttext.cc/docs/en/language-identification.html

En nuestra implementación, ya que el modelo solo puede clasificar pequeñas porciones de texto, no clasificamos el documento entero directamente, si no que vamos frase por frase.

Entonces, para cada frase en el documento, obtenemos el idioma con mayor probabilidad, calculada por el modelo. Con las frases clasificadas por idiomas, calculamos el porcentaje de carácteres en cada idioma. Finalmente, clasificamos el documento con la lengua que tenga más del 60% de carácteres. Cuando no hay ninguna lengua que sobrepase este 60%, los documentos se clasifican como "mx". En "mx" es normal encontrar documentos con lenguas minoritarias que el modelo no sabe clasificar bien (gallego, catalán) y que además contienen código o puntuación excesiva.

**Nosotros de momento vamos a aplicar fastext en el texto sin más, a la espera de que BSC nos pase los idiomas que ellos han detectado.**

Han hecho algo raro con los nuevos parquets que nos han pasado traducidos. Al hacer el extract de los que no han sido traducidos, muchos aparecen vacíos. Seguramente se puede resolver el fallo revisando el código de extracción, pero de momento vamos a trabajar haciendo un merge de los que ya teníamos bien procesado del dump anterior (``BSC_procesados``) con los nuevos procesados tras la traducción (``BSC_procesados_ca``).

**Había un error con lo que nos habían pasado traducidos en catalán. Nos solucionan el error y al mismo tiempo nos proporcionan la traducción de los pliegos en gallego y euskera.**

Correo de Jens: "Efectivamente había un problema de nuestro lado. Un script de preparación de datos que utilizamos antes de pasar el traductor ha cambiado el orden de los documentos y como resultado no todas las traducciones se habían incorporado correctamente en los ficheros Parquet.

Hemos cambiado la forma de combinar las traducciones con los Parquets existentes y el resultado parece correcto ahora. Hemos integrado también las traducciones de los textos en gallego y vasco. Los ficheros están subidos a nextp:PLACE/pdf2txt_all_translated y parecen incluir todas las traducciones:

documentos totales: **4114855**

documentos traducidos:
- ca: 166251
- gl: 25572
- eu: 2291

Cuidado que los campo de "alternative_lang" y "translated_content" para los documentos no traducidos puede tener el valor None / null, o puede ser un string vacío. Para evitar cualquier problema recomendamos mirar expresamente que "alternative_lang" sea "ca", "gl", o "eu" y usar "translated_content" solo en estos casos."

In [3]:
166251 + 25572 + 2291

194114

In [4]:
path_all = pathlib.Path("/export/data_ml4ds/NextProcurement/PLACE/BSC_procesados") # path to all
path_ca_tr = pathlib.Path("/export/data_ml4ds/NextProcurement/PLACE/Septiembre2024_procesados") # path to detected as Catalan and translated #BSC_procesados_ca #pdf2txt_ca_translated

In [5]:
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path) 

In [6]:
# Read ALL
paths = []
paths = [path for path in path_all.iterdir()]
all_dfs = []
for path_ in tqdm(paths):
    df = pd.read_parquet(path_)
    all_dfs.append(df)
combined_df = pd.concat(all_dfs)

100%|██████████| 17/17 [27:59<00:00, 98.79s/it] 


In [7]:
combined_df

,procurement_id,doc_name,extracted,extracted_tags,texto_heading,embeddings_heading
2,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297..."
10,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051..."
23,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05..."
35,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,íHJJ Agencia de I'Habitatge\n FUU de Catalunya...,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832..."
40,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294..."
...,...,...,...,...,...,...
250028,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204..."
250038,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086..."
250067,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,ANEX\nPliego de prescripciones técnicas que ha...,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037..."
250078,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800..."


In [8]:
# Read CA
paths = []
paths = [path for path in path_ca_tr.iterdir()]
all_dfs = []
for path_ in tqdm(paths):
    df = pd.read_parquet(path_)
    all_dfs.append(df)
combined_df_ca = pd.concat(all_dfs)

100%|██████████| 17/17 [07:14<00:00, 25.55s/it]


In [9]:
combined_df_ca

,procurement_id,doc_name,extracted,alternative_lang,translated
207,ntp00289491,ntp00289491_Pliego_Prescripciones_tecnicas_URI,arquitectos MERCADÉ. C/Nou DE SANTA TECLA 9 2o...,ca,yes
331,ntp00372307,ntp00372307_Pliego_Prescripciones_tecnicas_URI,Departamento de Seguridad Dirección de Adminis...,eu,yes
523,ntp00405990,ntp00405990_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Barcelona Movilidad e Infraest...,ca,yes
744,ntp00339949,ntp00339949_Pliego_Prescripciones_tecnicas_URI,¿;%'é/z Instituto de Formación Continua-IL3 E ...,ca,yes
781,ntp00297383,ntp00297383_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de La Pobla de Mafumet PLIEGO TÉC...,ca,yes
...,...,...,...,...,...
252598,ntp00727560,ntp00727560_Pliego_Prescripciones_tecnicas_URI,"SENÑA\nPROCEDIMIENTO ABIERTO PARA LA,ADJUDICAC...",,no
252615,ntp01088399,ntp01088399_Pliego_Prescripciones_tecnicas_URI,[\n Aulas para la formación de Empleo — FASE I...,,no
252625,ntp00596300,ntp00596300_Pliego_Prescripciones_tecnicas_URI,"Plaza de España, 1\n \n 13300 VALDEPEÑAS (Ciud...",,no
252630,ntp00860445,ntp00860445_Pliego_Prescripciones_tecnicas_URI,PPT Mantenimiento Instalaciones Contra incendi...,,no


In [10]:
combined_df_ca[(combined_df_ca.alternative_lang == "ca") | (combined_df_ca.alternative_lang == "eu") | (combined_df_ca.alternative_lang == "ga")]

,procurement_id,doc_name,extracted,alternative_lang,translated
207,ntp00289491,ntp00289491_Pliego_Prescripciones_tecnicas_URI,arquitectos MERCADÉ. C/Nou DE SANTA TECLA 9 2o...,ca,yes
331,ntp00372307,ntp00372307_Pliego_Prescripciones_tecnicas_URI,Departamento de Seguridad Dirección de Adminis...,eu,yes
523,ntp00405990,ntp00405990_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Barcelona Movilidad e Infraest...,ca,yes
744,ntp00339949,ntp00339949_Pliego_Prescripciones_tecnicas_URI,¿;%'é/z Instituto de Formación Continua-IL3 E ...,ca,yes
781,ntp00297383,ntp00297383_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de La Pobla de Mafumet PLIEGO TÉC...,ca,yes
...,...,...,...,...,...
252154,ntp01364258,ntp01364258_Pliego_Prescripciones_tecnicas_URI,"A...IUN'I”AMF,N'I""DF, FERRERIES Expediente: — ...",ca,yes
252399,ntp00393844,ntp00393844_Pliego_Prescripciones_tecnicas_URI,DOCUMENTO PLEC DE CLAUSULES (TD03-012): PLEC_P...,ca,yes
252439,ntp00009303,ntp00009303_Pliego_Prescripciones_tecnicas_URI,Cementerios de Barcelona cementerio es ciudad ...,ca,yes
252562,ntp01060556,ntp01060556_Pliego_Prescripciones_tecnicas_URI,Universidad de les Illes Balears Centro/Depart...,ca,yes


In [16]:
combined_df_ca[(combined_df_ca.alternative_lang == "ga")]

,procurement_id,doc_name,extracted,alternative_lang,translated


In [11]:
combined_df_ca.columns

Index(['procurement_id', 'doc_name', 'extracted', 'alternative_lang',
       'translated'],
      dtype='object')

In [13]:
combined_df_ca[combined_df_ca.translated == "yes"]

,procurement_id,doc_name,extracted,alternative_lang,translated
207,ntp00289491,ntp00289491_Pliego_Prescripciones_tecnicas_URI,arquitectos MERCADÉ. C/Nou DE SANTA TECLA 9 2o...,ca,yes
331,ntp00372307,ntp00372307_Pliego_Prescripciones_tecnicas_URI,Departamento de Seguridad Dirección de Adminis...,eu,yes
523,ntp00405990,ntp00405990_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Barcelona Movilidad e Infraest...,ca,yes
744,ntp00339949,ntp00339949_Pliego_Prescripciones_tecnicas_URI,¿;%'é/z Instituto de Formación Continua-IL3 E ...,ca,yes
781,ntp00297383,ntp00297383_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de La Pobla de Mafumet PLIEGO TÉC...,ca,yes
...,...,...,...,...,...
252154,ntp01364258,ntp01364258_Pliego_Prescripciones_tecnicas_URI,"A...IUN'I”AMF,N'I""DF, FERRERIES Expediente: — ...",ca,yes
252399,ntp00393844,ntp00393844_Pliego_Prescripciones_tecnicas_URI,DOCUMENTO PLEC DE CLAUSULES (TD03-012): PLEC_P...,ca,yes
252439,ntp00009303,ntp00009303_Pliego_Prescripciones_tecnicas_URI,Cementerios de Barcelona cementerio es ciudad ...,ca,yes
252562,ntp01060556,ntp01060556_Pliego_Prescripciones_tecnicas_URI,Universidad de les Illes Balears Centro/Depart...,ca,yes


In [12]:
merged_df = pd.merge(combined_df, combined_df_ca, on='doc_name', how='inner')

merged_df['extracted'] = merged_df.apply(
    lambda row: row['extracted_y'] if row['alternative_lang'] == 'ca' else row['extracted_x'], axis=1)

merged_df = merged_df.drop(columns=['extracted_x', 'extracted_y', 'procurement_id_y'])
merged_df

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...
3,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832...",,no,íHJJ Agencia de I'Habitatge\n FUU de Catalunya...
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...
...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...


In [13]:
merged_df["clean_extracted"] = merged_df["extracted"].apply(normalize_string)
merged_df = get_lang(merged_df, col_calculate_on = "clean_extracted")

-- Detecting language...
-- -- Language detect finished in 10821.378039121628


In [14]:
merged_df.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_29jul.parquet")

In [16]:
merged_df

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted,clean_extracted,lang
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn
3,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832...",,no,íHJJ Agencia de I'Habitatge\n FUU de Catalunya...,íHJJ Agencia de I'Habitatge FUU de Catalunya P...,cat_Latn
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn
...,...,...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn


In [15]:
sample_df = merged_df.sample(n=1000, random_state=42)
sample_df.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_29jul_sample1000.parquet")

In [17]:
merged_df_es = merged_df[merged_df.lang == "spa_Latn"]

In [18]:
merged_df_es

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted,clean_extracted,lang
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn
5,ntp01108524,ntp01108524_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.13603789, 0.22781396, -0.091998294, -0.10...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
...,...,...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn


In [19]:
merged_df_es.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_12aug_es.parquet")

In [20]:
merged_df_es

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted,clean_extracted,lang
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn
5,ntp01108524,ntp01108524_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.13603789, 0.22781396, -0.091998294, -0.10...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn
...,...,...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn


In [21]:
merged_df_es.columns

Index(['procurement_id_x', 'doc_name', 'extracted_tags', 'texto_heading',
       'embeddings_heading', 'alternative_lang', 'translated', 'extracted',
       'clean_extracted', 'lang'],
      dtype='object')

In [23]:
merged_df_es[["procurement_id_x","doc_name","translated", "extracted","clean_extracted", "lang"]].to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_12aug_es.parquet")

## Objectives extracted

In [24]:
import pandas as pd
df = pd.read_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_objectives/all_extracted_12aug_es.parquet")

In [25]:
df

,procurement_id_x,doc_name,translated,extracted,clean_extracted,lang,objective,in_text_score
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS S - CON...,spa_Latn,None,None
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn,None,None
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,spa_Latn,None,None
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,Ref: 50/088288.9/21 Agencia de Vivienda Social...,spa_Latn,None,None
5,ntp01108524,ntp01108524_Pliego_Prescripciones_tecnicas_URI,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,spa_Latn,None,None
...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,sura PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA...,spa_Latn,None,None
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,2020/02293/01 14/07/2020 ... COLEGIO DE INGENI...,spa_Latn,None,None
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,no,ANEX\nPliego de prescripciones técnicas que ha...,ANEX Pliego de prescripciones técnicas que ha ...,spa_Latn,None,None
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,a A S UNIVERSITAT POLITÉCNICA DE VALENCIA ... ...,spa_Latn,None,None


In [33]:
df.columns

Index(['procurement_id_x', 'doc_name', 'translated', 'extracted',
       'clean_extracted', 'lang', 'objective', 'in_text_score'],
      dtype='object')